In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


"""Change to the data folder"""
new_path = "../new_train/new_train/"
val_path = "../new_val_in/new_val_in/"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, training=True):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.training = training

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)

        return data


train_dataset  = ArgoverseDataset(data_path=new_path)
val_dataset = ArgoverseDataset(data_path=val_path,training=False)
#print((val_dataset[0]))
#print(len(train_dataset[0]))

### Create a loader to enable batch processing

In [3]:
batch_sz = 200

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = []
    out = []
    laneInfo = []
    city = []
#     print("pIn",batch[0]['p_in'])
    numbRows = 60
    for scene in batch:

        cityyy = numpy.zeros((60,1,4))

        if scene['city'] == 'PIT':
            cityyy[:,:,:] = 1

        city.append(numpy.dstack([cityyy]))
        lanes = numpy.zeros((numbRows * 19,2))
        lane_norm = numpy.zeros((numbRows * 19,2))

        lengthLane = min(numbRows * 19,len(scene['lane']))

        lanes[:lengthLane,:2] = scene['lane'][:lengthLane,:2]
        lane_norm[:lengthLane,:2] = scene['lane_norm'] [:lengthLane,:2]
        laneInfo.append(numpy.dstack([lanes.reshape(60,19,2),lane_norm.reshape(60,19,2)]))
        inp.append(numpy.dstack([scene['p_in'],scene['v_in']]))
        out.append(numpy.dstack([scene['p_out'], scene['v_out']]))

    laneInfo = torch.FloatTensor(laneInfo)
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    city = torch.FloatTensor(city)

    return [city,laneInfo,inp, out]

train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = True, collate_fn=my_collate, num_workers=8)

In [4]:
batch_sz = 2
def val_collate(batch):
    agentIds = []
    trackIds = []
    sceneIdxs = []
    laneInfo = []
    inp = []
    city = []
    numbRows = 60
    for scene in batch:
        """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
        agentIds.append(scene['agent_id'])
        trackIds.append(scene['track_id'])
        sceneIdxs.append(scene['scene_idx'])
        
        cityyy = numpy.zeros((60,1,4))
        #print(cityyy)
        if scene['city'] == 'PIT':
            cityyy[:,:,:] = 1
        city.append(numpy.dstack([cityyy]))
        
        lanes = numpy.zeros((numbRows * 19,2))
        lane_norm = numpy.zeros((numbRows * 19,2))
        lengthLane = min(numbRows * 19,len(scene['lane']))
        lanes[:lengthLane,:2] = scene['lane'][:lengthLane,:2]
        lane_norm[:lengthLane,:2] = scene['lane_norm'] [:lengthLane,:2]
        laneInfo.append(numpy.dstack([lanes.reshape(60,19,2),lane_norm.reshape(60,19,2)]))
        inp.append(numpy.dstack([scene['p_in'], scene['v_in']]))
    inp = torch.FloatTensor(inp)
    laneInfo = torch.FloatTensor(laneInfo)
    city = torch.FloatTensor(city)
    return [inp,sceneIdxs,agentIds,trackIds,laneInfo,city]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=val_collate, num_workers=6)

In [10]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class FullyConnectedModel(nn.Module):
    def __init__(self):
        super(FullyConnectedModel, self).__init__()
        

        self.hidden_dim = 240*32
        
        self.linear = nn.Sequential(
            nn.Linear(240 * 39, self.hidden_dim),
            nn.ReLU(), 

            nn.Linear(self.hidden_dim, 240*30)
        )

        
    def forward(self, x):
        x = self.linear(x)

        return x
    

### Training


In [11]:
from statistics import mean
import random
import numpy as np

torch.cuda.empty_cache()
agent_id = 0
learning_rate = 1e-5
momentum = 0.01
device = torch.device("cuda:0")
input_dim = 4 * 60    # input dimension
hidden_dim = 3000  # hidden layer dimension should be greater when batch_sz small
layer_dim = 3    # number of hidden layers
output_dim = 4   # output dimension
n_epochs = 30

'''Define Loss, Optimizer'''
#model = RNNModel(input_dim, output_dim, hidden_dim, layer_dim).to(device)
model = FullyConnectedModel()
# model.load_state_dict(torch.load('./3000Temp.pth'))
model = model.to(device)
# optimizer = optim.Adagrad(model.parameters(), lr=learning_rate,lr_decay=1e-6,weight_decay=1e-5)
# optimizer = optim.Adadelta(model.parameters())
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5,amsgrad=True)
# optimizer = optim.RMSprop(model.parameters(),lr=learning_rate,momentum=momentum)

model.train()
print("start")
loss_ema = -1
try:
    for i_epoch in range(n_epochs):
        for i_batch, sample_batch in enumerate(train_loader):
        #     print("test")
            city,laneInfo,inp, out = sample_batch
            laneInfo = laneInfo.cuda()
            inp = inp.cuda()
            out = out.cuda()
            city = city.cuda()
            optimizer.zero_grad()
            mixed = torch.cat([city,laneInfo,inp],2).transpose(1,2).reshape(-1,39,4 * 60).reshape(-1,39*240)
#             print(mixed.shape)
            
            y_pred = model(mixed)
            y_pred = y_pred.reshape((-1,30,60,4)).transpose(1,2)

            loss = (torch.mean((y_pred-out)**2))
            loss.backward()
            optimizer.step()
            if loss_ema < 0:
                loss_ema = loss
            loss_ema= loss_ema*0.90 +loss*0.1

            if i_batch % 20 == 0:
                print("epoch#{:d}".format(i_epoch),"batch#{:d}".format(i_batch),"scenes#{:d}".format(i_batch * batch_sz)," avg loss per scene(past 100): ",loss_ema.item(),loss.item())
        torch.save(model.state_dict(), './FCM_Adam_30Epochs_HDim7k_L2.pth')    
except KeyboardInterrupt:
    print("savedModel")
    torch.save(model.state_dict(), './FCM_Adam_30Epochs_HDim7k_l2.pth')


start
epoch#0 batch#0 scenes#0  avg loss per scene(past 100):  513607.21875 513607.21875
epoch#0 batch#20 scenes#40  avg loss per scene(past 100):  305466.09375 281830.5625
epoch#0 batch#40 scenes#80  avg loss per scene(past 100):  185492.359375 127041.3359375
epoch#0 batch#60 scenes#120  avg loss per scene(past 100):  126064.5703125 97269.203125
epoch#0 batch#80 scenes#160  avg loss per scene(past 100):  91952.609375 71166.7265625
epoch#0 batch#100 scenes#200  avg loss per scene(past 100):  74008.1796875 62660.203125
epoch#0 batch#120 scenes#240  avg loss per scene(past 100):  66202.546875 46758.078125
epoch#0 batch#140 scenes#280  avg loss per scene(past 100):  57086.8984375 89494.1171875
epoch#0 batch#160 scenes#320  avg loss per scene(past 100):  48920.75390625 64339.59375
epoch#0 batch#180 scenes#360  avg loss per scene(past 100):  46710.28125 32345.009765625
epoch#0 batch#200 scenes#400  avg loss per scene(past 100):  46124.265625 43800.35546875
epoch#0 batch#220 scenes#440  avg 

epoch#0 batch#1780 scenes#3560  avg loss per scene(past 100):  5439.12890625 4494.07666015625
epoch#0 batch#1800 scenes#3600  avg loss per scene(past 100):  5274.67578125 8862.5126953125
epoch#0 batch#1820 scenes#3640  avg loss per scene(past 100):  5730.57763671875 7550.2294921875
epoch#0 batch#1840 scenes#3680  avg loss per scene(past 100):  5696.919921875 10316.896484375
epoch#0 batch#1860 scenes#3720  avg loss per scene(past 100):  5899.94580078125 4391.091796875
epoch#0 batch#1880 scenes#3760  avg loss per scene(past 100):  5315.75390625 6353.970703125
epoch#0 batch#1900 scenes#3800  avg loss per scene(past 100):  5424.64892578125 8906.6953125
epoch#0 batch#1920 scenes#3840  avg loss per scene(past 100):  7105.87353515625 6876.0888671875
epoch#0 batch#1940 scenes#3880  avg loss per scene(past 100):  5715.71630859375 4195.2373046875
epoch#0 batch#1960 scenes#3920  avg loss per scene(past 100):  6618.3564453125 7553.47900390625
epoch#0 batch#1980 scenes#3960  avg loss per scene(past

epoch#0 batch#3480 scenes#6960  avg loss per scene(past 100):  2564.202392578125 1240.82958984375
epoch#0 batch#3500 scenes#7000  avg loss per scene(past 100):  1835.7344970703125 1450.6702880859375
epoch#0 batch#3520 scenes#7040  avg loss per scene(past 100):  1872.1429443359375 956.0230102539062
epoch#0 batch#3540 scenes#7080  avg loss per scene(past 100):  2408.343017578125 1117.9608154296875
epoch#0 batch#3560 scenes#7120  avg loss per scene(past 100):  2023.3994140625 1079.819091796875
epoch#0 batch#3580 scenes#7160  avg loss per scene(past 100):  1791.2041015625 1095.3878173828125
epoch#0 batch#3600 scenes#7200  avg loss per scene(past 100):  1899.6622314453125 3363.720703125
epoch#0 batch#3620 scenes#7240  avg loss per scene(past 100):  1626.1466064453125 1376.94677734375
epoch#0 batch#3640 scenes#7280  avg loss per scene(past 100):  2195.42626953125 5187.10693359375
epoch#0 batch#3660 scenes#7320  avg loss per scene(past 100):  2056.778076171875 2191.771728515625
epoch#0 batch#

epoch#0 batch#5160 scenes#10320  avg loss per scene(past 100):  1314.1204833984375 1201.952880859375
epoch#0 batch#5180 scenes#10360  avg loss per scene(past 100):  1328.1358642578125 1617.248291015625
epoch#0 batch#5200 scenes#10400  avg loss per scene(past 100):  1076.3359375 649.50048828125
epoch#0 batch#5220 scenes#10440  avg loss per scene(past 100):  1073.3328857421875 1394.17578125
epoch#0 batch#5240 scenes#10480  avg loss per scene(past 100):  865.1397705078125 626.4090576171875
epoch#0 batch#5260 scenes#10520  avg loss per scene(past 100):  1162.9984130859375 1319.921630859375
epoch#0 batch#5280 scenes#10560  avg loss per scene(past 100):  1204.8055419921875 1325.066650390625
epoch#0 batch#5300 scenes#10600  avg loss per scene(past 100):  1079.609619140625 462.0224914550781
epoch#0 batch#5320 scenes#10640  avg loss per scene(past 100):  1008.7918090820312 1135.3831787109375
epoch#0 batch#5340 scenes#10680  avg loss per scene(past 100):  1167.5673828125 1810.249755859375
epoch#

epoch#0 batch#6820 scenes#13640  avg loss per scene(past 100):  535.2201538085938 376.821044921875
epoch#0 batch#6840 scenes#13680  avg loss per scene(past 100):  686.204345703125 595.6282348632812
epoch#0 batch#6860 scenes#13720  avg loss per scene(past 100):  692.516357421875 459.7862548828125
epoch#0 batch#6880 scenes#13760  avg loss per scene(past 100):  1031.809326171875 1414.6357421875
epoch#0 batch#6900 scenes#13800  avg loss per scene(past 100):  680.1715087890625 1193.525390625
epoch#0 batch#6920 scenes#13840  avg loss per scene(past 100):  1282.8397216796875 440.2417907714844
epoch#0 batch#6940 scenes#13880  avg loss per scene(past 100):  675.4576416015625 347.0677185058594
epoch#0 batch#6960 scenes#13920  avg loss per scene(past 100):  778.73779296875 844.0562133789062
epoch#0 batch#6980 scenes#13960  avg loss per scene(past 100):  1274.7872314453125 570.2489013671875
epoch#0 batch#7000 scenes#14000  avg loss per scene(past 100):  682.3041381835938 288.2950439453125
epoch#0 

...

epoch#29 batch#13100 scenes#26200  avg loss per scene(past 100):  3.10800838470459 2.2154974937438965
epoch#29 batch#13120 scenes#26240  avg loss per scene(past 100):  3.0507469177246094 1.999515175819397
epoch#29 batch#13140 scenes#26280  avg loss per scene(past 100):  3.4940316677093506 4.792243957519531
epoch#29 batch#13160 scenes#26320  avg loss per scene(past 100):  3.042829990386963 2.3396010398864746
epoch#29 batch#13180 scenes#26360  avg loss per scene(past 100):  3.407120943069458 2.4491424560546875
epoch#29 batch#13200 scenes#26400  avg loss per scene(past 100):  3.2549450397491455 3.416897773742676
epoch#29 batch#13220 scenes#26440  avg loss per scene(past 100):  3.241344451904297 2.5838985443115234
epoch#29 batch#13240 scenes#26480  avg loss per scene(past 100):  4.4567131996154785 3.313040018081665
epoch#29 batch#13260 scenes#26520  avg loss per scene(past 100):  4.134156227111816 3.1630983352661133
epoch#29 batch#13280 scenes#26560  avg loss per scene(past 100):  3.512274

epoch#29 batch#14700 scenes#29400  avg loss per scene(past 100):  3.595008373260498 3.331605911254883
epoch#29 batch#14720 scenes#29440  avg loss per scene(past 100):  3.965217351913452 3.525632858276367
epoch#29 batch#14740 scenes#29480  avg loss per scene(past 100):  3.783290147781372 3.238558769226074
epoch#29 batch#14760 scenes#29520  avg loss per scene(past 100):  3.41312575340271 3.6660642623901367
epoch#29 batch#14780 scenes#29560  avg loss per scene(past 100):  3.426882743835449 3.215334892272949
epoch#29 batch#14800 scenes#29600  avg loss per scene(past 100):  3.5637521743774414 2.970832586288452
epoch#29 batch#14820 scenes#29640  avg loss per scene(past 100):  3.211902141571045 2.8656198978424072
epoch#29 batch#14840 scenes#29680  avg loss per scene(past 100):  10.939766883850098 3.6435530185699463
epoch#29 batch#14860 scenes#29720  avg loss per scene(past 100):  4.772313594818115 2.765106439590454
epoch#29 batch#14880 scenes#29760  avg loss per scene(past 100):  4.4142932891

epoch#29 batch#16300 scenes#32600  avg loss per scene(past 100):  2.868103265762329 2.566977024078369
epoch#29 batch#16320 scenes#32640  avg loss per scene(past 100):  3.1429169178009033 4.889692783355713
epoch#29 batch#16340 scenes#32680  avg loss per scene(past 100):  4.016469955444336 4.45850944519043
epoch#29 batch#16360 scenes#32720  avg loss per scene(past 100):  3.6210694313049316 4.494685173034668
epoch#29 batch#16380 scenes#32760  avg loss per scene(past 100):  4.103603363037109 5.746350288391113
epoch#29 batch#16400 scenes#32800  avg loss per scene(past 100):  3.743767023086548 2.5252604484558105
epoch#29 batch#16420 scenes#32840  avg loss per scene(past 100):  3.8970980644226074 6.7992987632751465
epoch#29 batch#16440 scenes#32880  avg loss per scene(past 100):  3.71331524848938 5.507810592651367
epoch#29 batch#16460 scenes#32920  avg loss per scene(past 100):  3.2033557891845703 2.7703299522399902
epoch#29 batch#16480 scenes#32960  avg loss per scene(past 100):  3.621982574

epoch#29 batch#17900 scenes#35800  avg loss per scene(past 100):  3.3518521785736084 2.8078432083129883
epoch#29 batch#17920 scenes#35840  avg loss per scene(past 100):  4.440492630004883 3.132030487060547
epoch#29 batch#17940 scenes#35880  avg loss per scene(past 100):  3.3608944416046143 2.8713788986206055
epoch#29 batch#17960 scenes#35920  avg loss per scene(past 100):  3.2563223838806152 2.827179431915283
epoch#29 batch#17980 scenes#35960  avg loss per scene(past 100):  3.005420446395874 3.572509288787842
epoch#29 batch#18000 scenes#36000  avg loss per scene(past 100):  3.4288082122802734 5.243131160736084
epoch#29 batch#18020 scenes#36040  avg loss per scene(past 100):  4.054974555969238 2.8999805450439453
epoch#29 batch#18040 scenes#36080  avg loss per scene(past 100):  3.841001272201538 4.009242057800293
epoch#29 batch#18060 scenes#36120  avg loss per scene(past 100):  3.514559030532837 2.785388469696045
epoch#29 batch#18080 scenes#36160  avg loss per scene(past 100):  3.7164716

epoch#29 batch#19500 scenes#39000  avg loss per scene(past 100):  3.9776687622070312 3.181422233581543
epoch#29 batch#19520 scenes#39040  avg loss per scene(past 100):  3.82430100440979 4.556271553039551
epoch#29 batch#19540 scenes#39080  avg loss per scene(past 100):  3.780224084854126 3.662017822265625
epoch#29 batch#19560 scenes#39120  avg loss per scene(past 100):  4.156805992126465 10.515210151672363
epoch#29 batch#19580 scenes#39160  avg loss per scene(past 100):  49.9848747253418 17.70724868774414
epoch#29 batch#19600 scenes#39200  avg loss per scene(past 100):  13.450899124145508 4.669369220733643
epoch#29 batch#19620 scenes#39240  avg loss per scene(past 100):  6.178195476531982 8.012353897094727
epoch#29 batch#19640 scenes#39280  avg loss per scene(past 100):  4.433567047119141 2.871377468109131
epoch#29 batch#19660 scenes#39320  avg loss per scene(past 100):  3.8314998149871826 4.297717094421387
epoch#29 batch#19680 scenes#39360  avg loss per scene(past 100):  4.706579208374

In [21]:
torch.save(model.state_dict(), './FCM_Adam_30Epochs_HDim7k_L2.pth')

In [12]:
import numpy as np
# import pandas as pd

save_file = "FCM_Adam_30Epochs_HDim7k_L2.csv"


header = ["ID"]
header += ["v"+str(x) for x in range(1, 61)]
print(header)

with open(save_file, 'w') as f:
    f.write(",".join(header)+"\n")

device = "cuda:0"

testmodel = FullyConnectedModel()
testmodel.load_state_dict(torch.load('FCM_Adam_30Epochs_HDim7k_L2.pth'))
testmodel.to(device)
testmodel.eval()

full_out = []
print("test")
varMaxOutput= [4773.,   4097.7,   193.19,  194.33]
varMinOutput=[ -53.912,    0.,    -210.04,  -187.71 ]
#         #Changed to 4 because we don't need lane and lane norm
batch_sz = 2
     
for i_batch, sample_batch in enumerate(val_loader):
    if i_batch % 100 == 0:
        print("batch #: ", i_batch * batch_sz)

    inp, scene_idx, agent_ids, track_ids,laneInfo,city = sample_batch
    city = city.cuda()
    laneInfo = laneInfo.cuda()
    inp = inp.cuda()

    mixed = torch.cat([city,laneInfo,inp],2).transpose(1,2).reshape(-1,39,4 * 60).reshape(-1,39*240)
    y_pred = testmodel(mixed)
    y_pred = y_pred.reshape((-1,30,60,4)).transpose(1,2)
    

    i_scene = 0
    out = []
    for ag_id in agent_ids:
        nestedTrackIds = track_ids[i_scene]
        j = 0
        for track_id in nestedTrackIds:

            if ag_id == track_id[0]:
                out.append(y_pred[i_scene][j][:,:2].reshape(60))
            j+=1
        i_scene+=1
                           
 

    with open(save_file, "a") as f:
        for i_scene in range(len(out)):
#             realOut = out[i].eval(session=tf.compat.v1.Session())
            array = out[i_scene]
#             print(out[i][0])
            row = ','.join([str(element.item()) for element in array])
            f.write(str(scene_idx[i_scene])+"," + row +"\n")


['ID', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v46', 'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v57', 'v58', 'v59', 'v60']
test
batch #:  0
batch #:  200
batch #:  400
batch #:  600
batch #:  800
batch #:  1000
batch #:  1200
batch #:  1400
batch #:  1600
batch #:  1800
batch #:  2000
batch #:  2200
batch #:  2400
batch #:  2600
batch #:  2800
batch #:  3000
